In [1]:
import pandas as pd
from preprocessing import preprocess

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

import numpy as np

import datetime

import re

In [45]:
class ClaimDataset(Dataset):

  def __init__(self, df):
    # TODO: keep claimid
    self.X = df.drop(columns=["sys_claimid","sys_fraud"]).to_numpy().astype('float32')
    self.y = df["sys_fraud"].to_numpy().astype('int64')
    self.IDs = df["sys_claimid"].to_numpy()
  
  def __len__(self):
    return len(self.X)

  def __getitem__(self, index):
    tensor = torch.tensor(self.X[index,:])
#     label = torch.LongTensor(self.y[index])
    label = self.y[index]
    ID = self.IDs[index]

    return tensor, label, ID
  
  def number_of_features(self):
    return self.X.shape[1]

In [22]:
class ClaimClassifier(nn.Module):
  def __init__(self, input_neurons):
    super(ClaimClassifier, self).__init__()
    
    #Linear
    self.classifier1 = nn.Linear(input_neurons, 32)
    
    #ReLU
    self.relu = nn.ReLU()
    
    #Linear
    self.classifier2 = nn.Linear(32, 2)


  def forward(self, tensor):
    
    x = self.classifier1(tensor)
    x = self.relu(x)
    logits = self.classifier2(x)

    return logits

In [51]:
def train_model(model, criterion, optimizer, train_dataloader, test_dataloader, device, epochs, max_patience):
  loss_values = []

  #Training
  print("- - T R A I N I N G - -")

  model_name = "{}.mdl".format(datetime.datetime.now().replace(microsecond=0).isoformat()).replace("/","_")
  patience = 0
  scores = []

  for epoch_i in range(epochs):

    print()
    print('> Epoch {:} of {:}'.format(epoch_i + 1, epochs))

    t0 = time.time()
    total_loss = 0
    
    for step, batch in enumerate(train_dataloader):

      if step % 1000 == 0 and not step == 0:
        print('├───Batch {:>3,} of {:>3,}. time={:.1f}s.'.format(step, len(train_dataloader)-1, time.time() - t0))
      if step == len(train_dataloader)-1:
        print('└───Batch {:>3,} of {:>3,}. time={:.1f}s.'.format(step, len(train_dataloader)-1, time.time() - t0))

      x = batch[0].to(device)
      y = torch.LongTensor(batch[1].to(device))
      #ID = batch[2].to(device)

      model.zero_grad()

      outputs = model(x)
      
      loss = criterion(outputs, y)
      total_loss += loss

      loss.backward()
      
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # TODO:check

      optimizer.step()
    
    avg_train_loss = total_loss / len(train_dataloader)            
    loss_values.append(avg_train_loss)

    print("")
    print("- Training loss: {0:.2f}".format(avg_train_loss))
    print("- Training Epoch took: {:.1f}s".format(time.time() - t0))
    
    # Validation

    print()
    print("- - V A L I D A T I N G - -")

    t0 = time.time()
    
    total_loss = 0

    model.eval()

    eval_loss, eval_accuracy, eval_auroc, eval_precision, eval_recall = 0, 0, 0, 0, 0
    nb_acc_steps, nb_auroc_steps, nb_eval_examples, nb_prc_steps, nb_rcl_steps = 0, 0, 0, 0, 0

    

    softmax = nn.Softmax(dim=1)
    
    eval_preds = np.array([])
    eval_probs = np.array([])
    eval_labels = np.array([])
    
    
    for batch in test_dataloader:
      x = batch[0].to(device)
      y = torch.LongTensor(batch[1].to(device))
      #ID = batch[2].to(device)

      with torch.no_grad():
        logits = model(x)
      
      loss = criterion(logits, y)

      total_loss += loss.item()
      
      probs = softmax(logits).detach().cpu().numpy()[:,1]
      logits = logits.detach().cpu().numpy()
      labels = y.to('cpu').numpy()
      preds = np.argmax(logits, axis=1).flatten()
      
      eval_preds = np.append(eval_preds, preds)
      eval_probs = np.append(eval_probs, probs)
      eval_labels = np.append(eval_labels, labels)
      
    
    eval_loss = total_loss / len(test_dataloader)
    
    print("")
    print("- Validation Accuracy: {0:.4f}".format(accuracy_score(eval_preds, eval_labels)))
    print("- Validation Precision: {0:.4f}".format(precision_score(eval_preds, eval_labels)))
    print("- Validation Recall: {0:.4f}".format(recall_score(eval_preds, eval_labels)))
    print("- Validation F1: {0:.4f}".format(f1_score(eval_preds, eval_labels)))
    print("- Validation AUROC: {0:.4f}".format(roc_auc_score(eval_labels, eval_probs)))
    print("- Validation loss: {0:.4f}".format(eval_loss))
    print("- Validation took: {:.1f}s".format(time.time() - t0))

    f1 = f1_score(eval_preds, eval_labels)
    # Save if new best score has been achieved
    if len(scores)==0:
      print("")
      print("Saving model...")
      torch.save(model.state_dict(), "savedmodels/{}".format(model_name))
    elif f1 >= max(scores):
      print("")
      print("New best F1 score, saving...")
      torch.save(model.state_dict(), "savedmodels/{}".format(model_name))
      patience = 0
    else:
      patience += 1
    
    
    if patience >= max_patience:
      "Maximum patience reached."
      break
    
    scores.append(f1)

    


  print("")
  print("Finished Training")
  print("")
  best_epoch = np.argmax(scores)
  print("Best seen F1 {:.4f} at epoch {}.".format(scores[best_epoch], best_epoch+1))

  print("Best model saved with filename: {}".format(model_name))
  print("Loading best model...")
  best_model = ClaimClassifier(train_dataloader.dataset.number_of_features())
  best_model.load_state_dict(torch.load("savedmodels/{}".format(model_name)))
  best_model.eval()
  print("Done.")

  return best_model #eval_accuracy/nb_eval_steps, eval_auroc/nb_eval_steps, eval_loss, 


In [34]:
# a = np.array([])
# np.append(a, np.array([1]))

In [35]:
train_file = "data/FRISS_ClaimHistory_training.csv"
test_file = "data/FRISS_ClaimHistory_test.csv"
fraud_file = "data/fraud_cases.csv"

train, test = preprocess(train_file, test_file, fraud_file)

Finished preprocessing.
Dropped 182 Train entries.
Dropped 43 Test entries.
Preprocessing took 1.14 seconds


In [36]:
# t = ClaimDataset(train)
# train_dataloader = DataLoader(t, batch_size = 12, num_workers = 2, shuffle=True)
# d = train_dataloader.dataset.number_of_features()
# d

In [52]:
epochs = 10
max_patience = 3

if torch.cuda.is_available():    
  DEVICE = torch.device("cuda")
  print('Using:', torch.cuda.get_device_name(0))
else:
  print('Using: CPU')
  DEVICE = torch.device("cpu")

torch.manual_seed(1004)

train_set = ClaimDataset(train)
test_set = ClaimDataset(test)
train_dataloader = DataLoader(train_set, batch_size = 8, num_workers = 2, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size = 8, num_workers = 2, shuffle=False)

input_features = train_set.number_of_features()

model = ClaimClassifier(input_features)
model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 3e-5, eps = 1e-8)

model = train_model(model, criterion, optimizer, train_dataloader, test_dataloader, DEVICE, epochs, max_patience)

Using: CPU
- - T R A I N I N G - -

> Epoch 1 of 10
├───Batch 1,000 of 9,977. time=1.4s.
├───Batch 2,000 of 9,977. time=2.7s.
├───Batch 3,000 of 9,977. time=4.0s.
├───Batch 4,000 of 9,977. time=5.5s.
├───Batch 5,000 of 9,977. time=6.9s.
├───Batch 6,000 of 9,977. time=8.5s.
├───Batch 7,000 of 9,977. time=10.0s.
├───Batch 8,000 of 9,977. time=11.9s.
├───Batch 9,000 of 9,977. time=14.1s.
└───Batch 9,977 of 9,977. time=16.4s.

- Training loss: 12.05
- Training Epoch took: 16.4s

- - V A L I D A T I N G - -

- Validation Accuracy: 0.9931
- Validation Precision: 0.0000
- Validation Recall: 0.0000
- Validation F1: 0.0000
- Validation AUROC: 0.3935
- Validation loss: 0.2603
- Validation took: 3.3s

Saving model...

> Epoch 2 of 10
├───Batch 1,000 of 9,977. time=1.9s.
├───Batch 2,000 of 9,977. time=3.7s.
├───Batch 3,000 of 9,977. time=5.5s.
├───Batch 4,000 of 9,977. time=7.4s.
├───Batch 5,000 of 9,977. time=9.4s.
├───Batch 6,000 of 9,977. time=11.0s.
├───Batch 7,000 of 9,977. time=12.7s.
├───Ba

In [ ]:
type()